# 1.THE PROJECT

This project is based on the Kaggle Dataset:
https://www.kaggle.com/shubhamchambhare/pokemons-and-there-stats

Created by https://www.kaggle.com/shubhamchambhare

It's a Dataset **with 1045 Pokemons** and these characteristics: *name, total, attack, defense, hit points, speed, special attack and special defense*

The idea is to create a predictive model to find out who would be the winner in a fight between two Pokemons.

For this I have carried out the following processes:

1- Transformation of original data by adding one more characteristic: **Category**, based on the score in the 'Total' column of the dataset.

2- Creation of a class called **Fight**. This class receives the index number of 2 pokemons from the dataset and returns the result of the combat.

  * The rules of combat are not those established in the cards game, but I have created a system based on the statistics provided by the database, in addition to a random component in relation to the percentages of damage from attacks and from defense blocking as well as percentages of probability of attacks and special defenses **(detailed explanation in section 2.5)**.
  * The result object has the quantitative differences between the values ​​of each Pokemon and the winner of the combat. Another parallel DF has the same data and the random statistics generated **(detailed explanation in section 2.5)**.

3- That object will be used to create our ML model, for this we create a dataset with a loop that will generate 186,750 combats. We will divide this Dataset into one with 40% of the confrontations to train and predict the models and the remaining 60% we will leave it unseen to evaluate the 3 best models **more information in sections 2.6 and 2.7**. This leaves us with a supervised learning binary classification problem.

4- We will carry out tests with different models and ensembles

5- We will evaluate the models based on several metrics and we will select the 3 best ones.

6- With the best 3 we will fit with the entire 40% training set and we will see the performance in two ways:
  * The accuracy of the model predicted at the unseen df (60% from Total combats)
  * We will do CrossValidation with the useen dataset (60% from Total combats. In Cv it will test n-times spliting the dataset n-times in training and test sets)
  * The winning model will be the one that obtains the best average between both values

7- With the winning model we will make a **fitting with the entire database** (60% unseen + 40% Training). We will simulate as if it was in production with a small interface through a form.

In this form we will also simulate as if the model itself learned with the new fights in such a way that every 30 fights it performs two actions:
  * Monitor with these new fights if the model continues to maintain good predictions; if not, it issues a warning.
  * New fitting to the model with the complete dataset and the new 30 predictions.

In addition there are two final appendices

**Appendix 1**: Code to tune and evaluate all the models at once

**Appendix 2**: Using the winning model, here is the code to make the predictions with the dataset that includes the random statistics generated in the combat. Although these statistics are impossible to know beforehand, it is a good exercise to observe **the importance of randomness** in the performance of the model.

**IMPORTANT**

**the selected models and parameter tunnig are by my choice. I understand that there may be other options. My best result is 88.64% accuracy. If someone wants to try and find better precision, it would be apreciated. In the Kaggle repositories I leave the databases created to download, in addition to being able to work with the same data**

**NOTE**
* Many of the variable names are in Spanish since initially it is the language in which I made it and changing them all thinking that there will not be a problem at some point is too optimistic. That is why I have translated the texts and prints, but not the names of variables, columns and indexes. I hope it is understood *

Thanks for your time

# 2.DATABASE CREATION

### 2.1 Libraries



In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import random 
from statistics import mean, stdev
import statistics
import time
import io
%matplotlib inline

### 2.2 Loading the Pokemon database

This project is based on the Kaggle Dataset: https://www.kaggle.com/shubhamchambhare/pokemons-and-there-stats

So that is the dataset that has to be loaded

In [ ]:
df = pd.read_csv('../input/pokemons-and-there-stats/pokemon.csv')

### 2.3 EDA

In [ ]:
#We take a first look at Pokemon's DB
df.info()

There are no null values.

There are 1045 Pokemons and all the characteristics are numeric except the name.

Let's now look at the first 3 Pokemons to see what the values ​​are like and check if there are duplicates for the creation of our combat database.

In [ ]:
df.head(3)

In [ ]:
df[df.duplicated()==True]

### 2.4 Data transformation

In [ ]:
# We remove duplicate values
df = df.drop_duplicates(['Name'], keep='last')
df.reset_index(drop=True, inplace=True)

In [ ]:
#We create the variable 'Categoria' according to the Total column. 
#We divide it into 6 categories with an equal proportion of Pokemon in each one.

quantiles = np.linspace(0,1,7)

bins = df['Total'].quantile(quantiles)
bins = bins.values

names = [1, 2, 3, 4, 5 , 6]

df['Categoria'] = pd.cut(df['Total'], bins=bins, labels=names)

df.head(3)

In [ ]:
#We check if the categories are evenly distributed
df.groupby(by='Categoria').count()
#This Dataset is available as a resource but this command allows you to save it locally

### 2.5 Creating the Fight Class:
  
  *This class delivers a combat object between two pokemons and suited for creating an ML model. Also delivers another object with the statistics associated with each combat for further analysis if desired*

The first object has the following characteristics:

* Difference between attack values ​​of both pokemons
* Difference between defense values ​​of both pokemons
* Difference between special attack values ​​of both pokemons
* Difference between the special defense values ​​of both pokemons
* The Pokemon that attacks first in binary class (0/1)
* Difference between category values ​​of both pokemons
* Winner of the match (0/1)


The second object is the same but adding the **associated combat statistics**: *average % of attack damage and defense block and how many times an attack or a special defense has occurred.*

The winner of the fight is obtained after applying different percentages to the characteristics of the pokemons **. The rules of combat are:

* The % of attack damage and defense block will always be between 50 and 100% of the base value.
* There is a 20% chance that a special attack or special defense will occur.
* If a special attack occurs, the damage and special defense values ​​will be between 30 and 100% of the base value.
* We assume that there is one attack per turn and the one with the highest speed begins. At the same speed, a random value of 50% will be applied each turn to see who starts.
* After each attack the opponent's pokemon's life is checked. if he is still alive it will be his turn to counterattack, if not, the result object is returned.

These calculations are performed in the implicit functions of the class:


  * **Combate**: It is the main function based on an infinite loop that stops when one of the Pokemons reaches 0 or less life. It is the one that calls the rest of the functions to obtain results or return the final object
  * **Enfrentamiento**: It is the one in charge within the combat function of calculating the damage that is produced in each turn for each pokemon. Returns the damage the percentages and aggregates of each confrontation.
  * **calcular_vida**: It is responsible for evaluating the amount of life left to the defending pokemon after applying the damage
  * **Agregar**: It is the final function that returns an object with the accumulated values ​​of the combat


** *The way of carrying out the fights is invented and they have nothing to do with the way of playing the pokemons cards, it is simply an invented format in order to create a database on which to work*

In [ ]:
class Fight:


    def __init__(self):
      self.turno = 1
## THIS IS THE MAIN FUNCTION, THE ONE THAT EXECUTES EVERYTHING AND RECEIVES AS VALUES THE INDICES OF 2 POKEMON

    def combate (self, pok1, pok2):
      self.pokemons = df.iloc[[pok1, pok2]][['Attack','Defence','Sp_attack', 'Sp_defence', 'Name']]
      self.vida = [df.iloc[pok1]['HP'],df.iloc[pok2]['HP']]
      self.velocidad = [df.iloc[pok1]['Speed'],df.iloc[pok2]['Speed']]
      self.estadisticas = pd.DataFrame(columns= ['porc_atac_pok1', 'porc_atac_pok2', 'porc_def_pok1', 
                                                 'porc_def_pok2','atac_esp_pok1', 'atac_esp_pok2', 
                                                 'def_esp_pok1', 'def_esp_pok2'])
      self.resultado = []
      self.pok1 = pok1
      self.pok2 = pok2

## THIS FUNCTION CALCULATES THE INDIVIDUAL MATCHES FOR EACH TURN (INCLUDING IN COMBAT)
      def enfrentamiento (atacante, defensor, pokemons):
        porc_atac = random.uniform(0.5,1)
        porc_defensa = random.uniform(0.5,1)
        probab_esp_atac = random.random()
        porc_atac_esp = random.uniform(0.3,1)
        probab_esp_def = random.random()
        porc_defensa_esp = random.uniform(0.3,1)
        atac_esp = 0
        def_esp = 0
        est_enfrent = []

        if probab_esp_atac <=0.20:
          daño = round((pokemons.iloc[atacante]['Attack']*porc_atac - pokemons.iloc[defensor]['Defence']*porc_defensa) + (porc_atac_esp*pokemons.iloc[atacante]['Sp_attack']), 2)
          atac_esp = 1
        else:
          daño =  round((pokemons.iloc[atacante]['Attack']*porc_atac - pokemons.iloc[defensor]['Defence']*porc_defensa), 2)
        
        if probab_esp_def <=0.20:
          daño = daño - round((pokemons.iloc[defensor]['Sp_defence']*porc_defensa_esp),2)
          def_esp = 1
    
        est_enfrent = [porc_atac, porc_defensa, atac_esp, def_esp]
        return daño, est_enfrent


#################################################################################

## THIS FUNCTION CALCULATES HOW MUCH LIFE IS REMAINING TO THE POKEMON THAT HAS BEEN DEFENDED (INCLUDED IN COMBAT)

      def calcularvida (daño, vida, atacante, defensor, pokemons):
        if daño >=0:
          vida[defensor] = round(vida[defensor] - daño,2)
             
        return vida[defensor]

###################################################################################

## WITH THIS FUNCTION WE CREATE THE OBJECT THAT WILL FEED THE DATABASE OF POKEMONS BATTLES (INCLUDED IN COMBAT)

      def agregar (pok1, pok2, dif_vel, ganador, estadisticas):
        dif_atac = df.iloc[pok1]['Attack']-df.iloc[pok2]['Attack']
        dif_def = df.iloc[pok1]['Defence']-df.iloc[pok2]['Defence']
        dif_esp_atc = df.iloc[pok1]['Sp_attack']-df.iloc[pok2]['Sp_attack']
        dif_esp_def = df.iloc[pok1]['Sp_defence']-df.iloc[pok2]['Sp_defence']
        dif_categ = df.iloc[pok1]['Categoria']-df.iloc[pok2]['Categoria']
        ganador = ganador
        dif_vel = dif_vel
        #ADDING THE 2 OBJECTS
        agregado = pd.DataFrame({'Dif Atac':[dif_atac], 'Dif Def':[dif_def], 'Dif atac espe':[dif_esp_atc], 
                                 'Dif def esp':[dif_esp_def], 'Ataca primero':[dif_vel], 
                                 'Dif Categ': [dif_categ],'Ganador': ganador })
        
        agregado_st = pd.DataFrame({'Dif Atac':[dif_atac], 'Dif Def':[dif_def], 'Dif atac espe':[dif_esp_atc], 
                                 'Dif def esp':[dif_esp_def], 'Ataca primero':[dif_vel], 
                                 'Dif Categ': [dif_categ], 'porc_atac_pok1':estadisticas['porc_atac_pok1'] , 
                                 'porc_atac_pok2':estadisticas['porc_atac_pok2'] , 
                                'porc_def_pok1':estadisticas['porc_def_pok1'] ,'porc_def_pok2':estadisticas['porc_def_pok2'], 
                                'atac_esp_pok1':estadisticas['atac_esp_pok1'] ,'atac_esp_pok2':estadisticas['atac_esp_pok2'] , 
                                'def_esp_pok1':estadisticas['def_esp_pok1'] ,'def_esp_pok2':estadisticas['def_esp_pok2'],
                                 'Ganador': ganador})
        
        
        
        return agregado,agregado_st 

# WE CREATE AN INFINITE LOOP FOR THE FIGHT FUNCTION TO BE RUNNING UNTIL A POKEMON REACHED 0 OR LESS HP POINTS
      while True:

        
        self.turno +=1
   
# SPEEDS ARE COMPARED
# IN THIS CASE, POKEMON 1 IS FASTER THAN POKEMON 2
#####################################################################

        if self.velocidad[0] > self.velocidad[1]:
            # We call the enfrentamiento function passing the appropriate parameters taking into account that it attacks pok1 and defends pok2
            dif_vel = 0
            atacante = 0
            defensor = 1
            datos_enfrent = enfrentamiento(atacante, defensor, self.pokemons)
            #calculating the damage
            daño = datos_enfrent[0]
            #Adding combat statistics
            self.estadisticas = self.estadisticas.append({'porc_atac_pok1':datos_enfrent[1][0], 'porc_def_pok2':datos_enfrent[1][1],
                   'atac_esp_pok1':datos_enfrent[1][2], 'def_esp_pok2':datos_enfrent[1][3]}, ignore_index=True)

            # Calculating remaining life and check status
            self.vida[defensor] = calcularvida(daño, self.vida, atacante, defensor, self.pokemons)
            if self.vida[defensor] <=0:
              medias = self.estadisticas.iloc[:,[0,1,2,3]].mean()
              sumas = self.estadisticas.iloc[:,[4,5,6,7]].apply(np.sum)
              estadisticas = {'porc_atac_pok1':medias[0] , 'porc_atac_pok2':medias[1] , 
                                'porc_def_pok1':medias[2] ,'porc_def_pok2':medias[3],
                                'atac_esp_pok1':sumas[0] ,'atac_esp_pok2':sumas[1] , 
                                'def_esp_pok1':sumas[2] ,'def_esp_pok2':sumas[3] }
              resultado = agregar(self.pok1, self.pok2, dif_vel, atacante, estadisticas)
             
              return  resultado
              break

            #Pokemon's defender turn
            
            atacante = 1
            defensor = 0
            # We call the enfrentamiento function passing the appropriate parameters 
            # taking into account that pok2 attacks and defends pok1
            datos_enfrent = enfrentamiento(atacante, defensor, self.pokemons)
            #calculating the damage
            daño = datos_enfrent[0]
            #Adding combat statistics
            self.estadisticas = self.estadisticas.append({'porc_atac_pok2':datos_enfrent[1][0], 'porc_def_pok1':datos_enfrent[1][1],
                   'atac_esp_pok2':datos_enfrent[1][2], 'def_esp_pok1':datos_enfrent[1][3]}, ignore_index=True)
            
            # Calculating remaining life and check status
            self.vida[defensor] = calcularvida(daño, self.vida, atacante, defensor, self.pokemons)
             
            if self.vida[defensor] <=0:
              medias = self.estadisticas.iloc[:,[0,1,2,3]].mean()
              sumas = self.estadisticas.iloc[:,[4,5,6,7]].apply(np.sum)
              estadisticas = {'porc_atac_pok1':medias[0] , 'porc_atac_pok2':medias[1] , 
                                'porc_def_pok1':medias[2] ,'porc_def_pok2':medias[3],
                                'atac_esp_pok1':sumas[0] ,'atac_esp_pok2':sumas[1] , 
                                'def_esp_pok1':sumas[2] ,'def_esp_pok2':sumas[3] }
              resultado = agregar(self.pok1, self.pok2, dif_vel, atacante, estadisticas)
                
              return  resultado
              break

# WE COMPARE SPEEDS, THIS CASE BOTH ARE THE SAME AND A RANDOM VALUE IS GIVEN TO START
#################################################################################################
        elif self.velocidad[0] == self.velocidad[1]:
            num = random.random()
            
            if num <= 0.5:
## IN THIS CASE WE ASSUME THAT POKEMON 1 HAS BEEN FASTER
            
                atacante = 0
                defensor = 1
                dif_vel = 0
                # We call the enfrentamiento function passing the appropriate parameters taking into account 
                # that pok1 attacks and pok2 defends
                datos_enfrent = enfrentamiento(atacante, defensor, self.pokemons)
                #calculating the damage
                daño = datos_enfrent[0]
                #Adding combat statistics
                self.estadisticas = self.estadisticas.append({'porc_atac_pok1':datos_enfrent[1][0], 'porc_def_pok2':datos_enfrent[1][1],
                   'atac_esp_pok1':datos_enfrent[1][2], 'def_esp_pok2':datos_enfrent[1][3]}, ignore_index=True)
                # Calculating remaining life and check status
                self.vida[defensor] = calcularvida(daño, self.vida, atacante, defensor, self.pokemons)

                if self.vida[defensor] <=0:
                  medias = self.estadisticas.iloc[:,[0,1,2,3]].mean()
                  sumas = self.estadisticas.iloc[:,[4,5,6,7]].apply(np.sum)
                  estadisticas = {'porc_atac_pok1':medias[0] , 'porc_atac_pok2':medias[1] , 
                                'porc_def_pok1':medias[2] ,'porc_def_pok2':medias[3],
                                'atac_esp_pok1':sumas[0] ,'atac_esp_pok2':sumas[1] , 
                                'def_esp_pok1':sumas[2] ,'def_esp_pok2':sumas[3] }

                  resultado = agregar(self.pok1, self.pok2,dif_vel, atacante, estadisticas)
                    
                  return  resultado
                  break
                #Pokemon's defender turn
            
                atacante = 1
                defensor = 0
            
                # We call the enfrentamiento function passing the appropriate parameters 
                # taking into account that pok2 attacks and defends pok1
                datos_enfrent = enfrentamiento(atacante, defensor, self.pokemons)
                #calculating the damage
                daño = datos_enfrent[0]
                #Adding combat statistics
                self.estadisticas = self.estadisticas.append({'porc_atac_pok2':datos_enfrent[1][0], 'porc_def_pok1':datos_enfrent[1][1],
                   'atac_esp_pok2':datos_enfrent[1][2], 'def_esp_pok1':datos_enfrent[1][3]}, ignore_index=True)
                
                # Calculating remaining life and check status
                self.vida[defensor] = calcularvida(daño, self.vida, atacante, defensor, self.pokemons)   
            
                if self.vida[defensor] <=0:
                    medias = self.estadisticas.iloc[:,[0,1,2,3]].mean()
                    sumas = self.estadisticas.iloc[:,[4,5,6,7]].apply(np.sum)
                    estadisticas = {'porc_atac_pok1':medias[0] , 'porc_atac_pok2':medias[1] , 
                                'porc_def_pok1':medias[2] ,'porc_def_pok2':medias[3],
                                'atac_esp_pok1':sumas[0] ,'atac_esp_pok2':sumas[1] , 
                                'def_esp_pok1':sumas[2] ,'def_esp_pok2':sumas[3] }

                    resultado = agregar(self.pok1, self.pok2,dif_vel, atacante, estadisticas)
                    
                    return  resultado
                    break
          
            else:
## IN THIS CASE WE ASSUME THAT POKEMON 2 HAS BEEN FASTER

                atacante = 1
                defensor = 0
                dif_vel = 1

                # We call the enfrentamiento function passing the appropriate parameters 
                # taking into account that pok2 attacks and defends pok1
                datos_enfrent = enfrentamiento(atacante, defensor, self.pokemons)
                #calculating the damage
                daño = datos_enfrent[0]
                #Adding combat statistics
                self.estadisticas = self.estadisticas.append({'porc_atac_pok2':datos_enfrent[1][0], 'porc_def_pok1':datos_enfrent[1][1],
                   'atac_esp_pok2':datos_enfrent[1][2], 'def_esp_pok1':datos_enfrent[1][3]}, ignore_index=True)
                
                # Calculating remaining life and check status
                self.vida[defensor] = calcularvida(daño, self.vida, atacante, defensor, self.pokemons)
           
                if self.vida[defensor] <=0:
                    medias = self.estadisticas.iloc[:,[0,1,2,3]].mean()
                    sumas = self.estadisticas.iloc[:,[4,5,6,7]].apply(np.sum)
                    estadisticas = {'porc_atac_pok1':medias[0] , 'porc_atac_pok2':medias[1] , 
                                'porc_def_pok1':medias[2] ,'porc_def_pok2':medias[3],
                                'atac_esp_pok1':sumas[0] ,'atac_esp_pok2':sumas[1] , 
                                'def_esp_pok1':sumas[2] ,'def_esp_pok2':sumas[3] }

                    resultado = agregar(self.pok1, self.pok2,dif_vel, atacante, estadisticas)
                    
                    return  resultado
                    break

                ##Pokemon's defender turn
            
                atacante = 0
                defensor = 1
                # We call the enfrentamiento function passing the appropriate parameters taking into account 
                # that it attacks pok1 and defends pok2
                datos_enfrent = enfrentamiento(atacante, defensor, self.pokemons)
                #calculating the damage
                daño = datos_enfrent[0]
                #Adding combat statistics
                self.estadisticas = self.estadisticas.append({'porc_atac_pok1':datos_enfrent[1][0], 'porc_def_pok2':datos_enfrent[1][1],
                   'atac_esp_pok1':datos_enfrent[1][2], 'def_esp_pok2':datos_enfrent[1][3]}, ignore_index=True)
                # Calculating remaining life and check status
                self.vida[defensor] = calcularvida(daño, self.vida, atacante, defensor, self.pokemons)
                                
                if self.vida[defensor] <=0:
                    medias = self.estadisticas.iloc[:,[0,1,2,3]].mean()
                    sumas = self.estadisticas.iloc[:,[4,5,6,7]].apply(np.sum)
                    estadisticas = {'porc_atac_pok1':medias[0] , 'porc_atac_pok2':medias[1] , 
                                'porc_def_pok1':medias[2] ,'porc_def_pok2':medias[3],
                                'atac_esp_pok1':sumas[0] ,'atac_esp_pok2':sumas[1] , 
                                'def_esp_pok1':sumas[2] ,'def_esp_pok2':sumas[3] }

                    resultado = agregar(self.pok1, self.pok2,dif_vel, atacante, estadisticas)
                    
                    return  resultado
                    break

# IN THIS CASE, POKEMON 2 IS FASTER THAN POKEMON 1
################################################################################

        else:
            atacante = 1
            defensor = 0
            dif_vel = 1

            # We call the enfrentamiento function passing the appropriate parameters 
            # taking into account that pok2 attacks and defends pok1
            datos_enfrent = enfrentamiento(atacante, defensor, self.pokemons)
            #calculating the damage
            daño = datos_enfrent[0]
            #Adding combat statistics
            self.estadisticas = self.estadisticas.append({'porc_atac_pok2':datos_enfrent[1][0], 'porc_def_pok1':datos_enfrent[1][1],
                   'atac_esp_pok2':datos_enfrent[1][2], 'def_esp_pok1':datos_enfrent[1][3]}, ignore_index=True)
            # Calculating remaining life and check status
            self.vida[defensor] = calcularvida(daño, self.vida, atacante, defensor, self.pokemons)
           
            if self.vida[defensor] <=0:
                medias = self.estadisticas.iloc[:,[0,1,2,3]].mean()
                sumas = self.estadisticas.iloc[:,[4,5,6,7]].apply(np.sum)
                estadisticas = {'porc_atac_pok1':medias[0] , 'porc_atac_pok2':medias[1] , 
                                'porc_def_pok1':medias[2] ,'porc_def_pok2':medias[3],
                                'atac_esp_pok1':sumas[0] ,'atac_esp_pok2':sumas[1] , 
                                'def_esp_pok1':sumas[2] ,'def_esp_pok2':sumas[3] }

                resultado = agregar(self.pok1, self.pok2,dif_vel, atacante, estadisticas)
                
                return  resultado
                break
                

            #Pokemon's defender turn
            
            atacante = 0
            defensor = 1
            # We call the enfrentamiento function passing the appropriate parameters taking into account 
            # that it attacks pok1 and defends pok2
            datos_enfrent = enfrentamiento(atacante, defensor, self.pokemons)
            ##calculating the damage
            daño = datos_enfrent[0]
            #Adding combat statistics
            self.estadisticas = self.estadisticas.append({'porc_atac_pok1':datos_enfrent[1][0], 'porc_def_pok2':datos_enfrent[1][1],
                   'atac_esp_pok1':datos_enfrent[1][2], 'def_esp_pok2':datos_enfrent[1][3]}, ignore_index=True)
            # Calculating remaining life and check status
            self.vida[defensor] = calcularvida(daño, self.vida, atacante, defensor, self.pokemons)

            if self.vida[defensor] <=0:
                medias = self.estadisticas.iloc[:,[0,1,2,3]].mean()
                sumas = self.estadisticas.iloc[:,[4,5,6,7]].apply(np.sum)
                estadisticas = {'porc_atac_pok1':medias[0] , 'porc_atac_pok2':medias[1] , 
                                'porc_def_pok1':medias[2] ,'porc_def_pok2':medias[3],
                                'atac_esp_pok1':sumas[0] ,'atac_esp_pok2':sumas[1] , 
                                'def_esp_pok1':sumas[2] ,'def_esp_pok2':sumas[3] }

                resultado = agregar(self.pok1, self.pok2,dif_vel, atacante, estadisticas)
                
                return  resultado
                break

In [ ]:
## We check two objects are returned
test = Fight().combate(1,150)
test[0]

In [ ]:
test[1]

### 2.6 Creation of the combat database:
* We created a sample of **250 Pokemons** not to have an excessively long database.
* Then they will fight all the pokemons against all and we will repeat it 3 times. In total **they will face each other 6 times** to give us various values of the same combat. This assumes **250 x 249 x 3 = 186,750 data** of clashes.

Due to the **randomness of certain parameters** in the confrontations, the winner can vary in each fight, especially in those that present similar values ​​in certain characteristics. For this reason, when creating the database, 6 fights are established between each pokemon.

On the other hand **the statistics created in each combat are saved together with the prediction aggregates**. Although they are not known beforehand and do not serve as a prediction model, they can be used to see if they are relevant in who is the winner in a fight if a posterior analysis is desired.

In [ ]:
df_combates_pokemons = pd.DataFrame()
df_estadísticas_combate = pd.DataFrame()
sample = df.sample(n=250, random_state=14)

#Checking for balanced categories
print(sample.groupby(by='Categoria').count())

#Creating the combtes object which will call the combat function of Fight class
combates = Fight()

In [ ]:
#Combat database creating process. Lasts an hour and a half on Google Collab so we directly
# WE SKIP THIS PROCESS FOR TIME SAVING AT COMMITING TIME
""" --- SKIPPING
start = time.time()

for v in range(3):
    for i in sample.index:
      for u in sample.index:
        if u != i:
          df_combates_pokemons = df_combates_pokemons.append(combates.combate(i,u)[0], ignore_index=True)
          df_estadísticas_combate = df_estadísticas_combate.append(combates.combate(i,u)[1], ignore_index=True)
stop = time.time()-start
print("Time dedicated to create the Dataset: ", stop/60 , " minutes")

--- END CODE SKIPPED"""

In [ ]:
#We mix it randomly and prevent the index from becoming a column with reset_index
# WE ALSO SKIP THIS PROCESSES AS THEY ARE ALL DONE IN THE DATABASES TO DOWNLOAD
""" ---- SKIPPING THIS 
df_combates_pokemons = df_combates_pokemons.sample(frac=1).reset_index(drop=True)
#We mix it a second time
from sklearn.utils import shuffle
df_combates_pokemons = shuffle(df_combates_pokemons)

#This Dataset will be available as resources but this command allows you to save it locally
df_combates_pokemons.to_csv('Pokemons_combate_full_250_pokemons.csv', index=False)

--- END CODE SKIPPED"""

*The process of creating the database with the three for loops, lasts an hour and a half. To avoid this, you can directly download the files:*

  **Pokemons_combate_full_250_pokemons.csv**
  
  **Estadisticas_combate.csv**
  

In [ ]:
df_combates_pokemons = pd.read_csv('../input/full-pokemons-combat-dataset/Pokemons_combate_full_250_pokemons.csv')
df_estadísticas_combate = pd.read_csv('../input/full-combat-dataset-with-statistics/Estadisticas_combate.csv')

In [ ]:
df_combates_pokemons.head()

In [ ]:
print(df_combates_pokemons.info(), "\n\n\n", df_combates_pokemons.describe())

### 2.7 DB for training and testing creation:
* We create the database to work with 40% of the fights
* We will use the remaining 60% to see how the best models behave with unseen data, such as a second test or validation dataset. It won´t be used until the end of the model selection process.


## 2.8 Combat DF EDA

In [ ]:
split = int(len(df_combates_pokemons)*.4)
df_combates_pokemons_train = df_combates_pokemons[:split]
df_combates_pokemons_valid = df_combates_pokemons[split+1:]
len(df_combates_pokemons_valid), len(df_combates_pokemons_train)

In [ ]:
df_combates_pokemons_train.describe()

In [ ]:
# We check that there are important correlations between the winner and the difference in category, attack and defense.
sns.heatmap(data= df_combates_pokemons_train.corr(), annot=True)

In [ ]:
#As we expected the winners are balanced
sns.countplot(data=df_combates_pokemons_train, x=df_combates_pokemons_train['Ganador'])

In [ ]:
# We check that the category difference is an important factor as the difference is greater.
#Seems a good starting point for separating nodes in decision trees
sns.histplot(data= df_combates_pokemons_train, x=df_combates_pokemons_train['Dif Categ'], hue=df_combates_pokemons_train['Ganador'],stat='count')

In [ ]:
#Attacking first seems to give some advantage
sns.histplot(data= df_combates_pokemons_train, x=df_combates_pokemons_train['Ataca primero'], hue=df_combates_pokemons_train['Ganador'],stat='count')

In [ ]:
# Although it seems that there are some outliers, it is quite clear that there are models 
#that should work well as Knn or LogisticRegression, even XGBoost.
sns.pairplot(df_combates_pokemons_train, hue='Ganador')

# 3.TESTING WITH MODELS
### Importing necessary libraries and resources


In [ ]:
## This library allows to use BayesSearchCV, a bayesian grid searcher
!pip install scikit-optimize

In [ ]:
import tensorflow_probability as tfp
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold, RepeatedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix, roc_auc_score, brier_score_loss
from sklearn.pipeline import Pipeline, make_pipeline
import tensorflow.keras as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
import joblib

## 3.1 TESTING WITH BASIC MODELS



## Creation of sets, functions and generic comparison tables
* Creation of the training and test set
* Creation of function to tune the desired models with BayesSearchCV.
* Creation the model's DF which will include all the models after tunning for the best parameters.
* The  model's DF contains the best model after the optimization of hyperparameters and the column 'scaler' that shows if it is necessary to scale the data before doing the fit and predict.
* We also create a table that we will use later to compare models and built with these values:
  * *Model precision, Training precision, Test Cross_val_score, ROC-AUC score, CV Std, Brier Loss, fit execution time and predict execution time*
  
  This table will be used later in the **EvalMod** function, that returns the evaluation metrics once the different models have passed through it.

  **Note**

*Although usual is to make first CV to validate the model and then look for the parameters, the decision to do it the opposite is the following*:

*According to the data, it seems that some characteristics are distributed following a Gaussian function, so it is to be expected that all the classification models behave well, so we use the CV after looking for the parameters as another valuation measure when choosing the optimal model.*


In [ ]:
X = df_combates_pokemons_train.drop('Ganador',axis=1)
y = df_combates_pokemons_train['Ganador']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=14)

#We scale the values for the models that need it
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [ ]:
#Creation of the Dataframes to pass through the evaluation function

modelos = pd.DataFrame(columns=['modelo','scaler'])
comparativa_modelos = pd.DataFrame(columns=['Modelo', 'Prec_test', 'Prec_train',
                                            'Prec_cv','ROC-AUC','STD_cv','Brier_loss',
                                            'T_ejec_fit', 'T_ejec_pred'])

In [ ]:
# Creation of the TunningModel function 
# It searchs for the best hyperparameters in the selected models if required

def TuningModel(modelo, parametros, X_train, y_train, verbose=1, scoring='accuracy'):
    
    start = time.time()
    mod_bayes_grid = BayesSearchCV(modelo, parametros, verbose=verbose, scoring=scoring)
    mod_bayes_grid.fit(X_train, y_train)
    stop = time.time()
    print("Tiempo de entrenamiento del Bayesian Search: ", (stop - start), " segundos")

    return mod_bayes_grid.best_estimator_

**Thought the whole tunning could be done with a for loop or even Pipeline in few lines of code, I prefer to go one by one just to show the best models each time. At Appendix 1 there is code to do the whole process at once.**

### KNN
  * We use the values transformed with StandardScaler

In [ ]:
mod_Knn = KNeighborsClassifier()
vecinos = np.arange(5,71,5)
parametros = {
    'n_neighbors':vecinos,
    'weights': ['uniform','distance'], 
    'leaf_size':np.linspace(1,10,10).astype(int)
    }

In [ ]:
mod_Knn = TuningModel(mod_Knn,parametros, X_train_sc, y_train)

In [ ]:
mod_Knn

In [ ]:
modelos = modelos.append({'modelo':mod_Knn, 'scaler':True}  , ignore_index=True)

### SGD
  * We use the values transformed with StandardScaler


In [ ]:
mod_sgd = SGDClassifier(loss='modified_huber')
penalty = ['l1', 'l2', 'elasticnet'] 
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000] 
learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive'] 
eta0 = [1, 10, 100] 
parametros = dict(penalty=penalty, 
alpha=alpha, 
learning_rate=learning_rate,
eta0=eta0) 


In [ ]:
mod_sgd = TuningModel(mod_sgd ,parametros, X_train_sc, y_train)

In [ ]:
mod_sgd

In [ ]:
modelos = modelos.append({'modelo':mod_sgd, 'scaler':True}  , ignore_index=True)

### LOGISTIC REGRESSION

In [ ]:
mod_lr = LogisticRegression()
parametros = {'C': [ 1, 50, 100, 150], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'max_iter':[100, 150,200]}

In [ ]:
mod_lr = TuningModel(mod_lr,parametros, X_train_sc, y_train)

In [ ]:
mod_lr 

In [ ]:
modelos = modelos.append({'modelo':mod_lr, 'scaler':True}  , ignore_index=True)

### LIGHT GRADIENT BOOST CLASSIFIER

In [ ]:
modL = LGBMClassifier(objective='binary', n_estimators=150 )
parametros = {'num_leaves': [31,50,70],
              'learning_rate':[0.1,0.05,0.001],
              'max_depth':[-1,7,10,30]
}

In [ ]:
modL = TuningModel(modL,parametros, X_train, y_train)

In [ ]:
modL

In [ ]:
modelos = modelos.append({'modelo':modL, 'scaler':False}  , ignore_index=True)

## 3.2 TESTING ENSEMBLE MODELS AN NEURONAL NETWORK

First we tested ensemble models like XGBoost or ADAboost and later Sklearn Stack models like vote classifier. Then we add a neural network and Vstack.



 ### XGBOOST

In [ ]:
mod_xgb = XGBClassifier(n_estimators=500)
parametros = {'colsample_bylevel': [0.1,0.3,0.8,1],
              'colsample_bynode': [0.3,0.5,1],
              'gamma' : [0.05, 0.1, 0.2, 0.5, 1],
              'learning_rate': [0.005, 0.02,0.05],
              'max_depth': [1,2,4,6],
              'min_child_weight': [1,5,6,9],
              'subsample': [0.5,0.8,1]        
              }

In [ ]:
mod_xgb = TuningModel(mod_xgb,parametros, X_train, y_train)

In [ ]:
mod_xgb

In [ ]:
modelos = modelos.append({'modelo':mod_xgb, 'scaler':False}  , ignore_index=True)

### ADABOOST

In [ ]:
mod_adaboost = AdaBoostClassifier(n_estimators=150, random_state=14)
parametros = {     
    'learning_rate': [0.05,0.1,0.5,0.8]
}

In [ ]:
mod_adaboost = TuningModel(mod_adaboost,parametros, X_train, y_train)

In [ ]:
mod_adaboost

In [ ]:
modelos = modelos.append({'modelo':mod_adaboost, 'scaler':False}  , ignore_index=True)

### RANDOM FOREST

In [ ]:
mod_rndForest = RandomForestClassifier(verbose=3, n_jobs=-1, random_state=14)
max_depth = [int(x) for x in np.linspace(5, 35, num = 10)]
max_depth.append(None)
parametros = {
    'n_estimators': [100, 200],
    'max_depth': max_depth
}

In [ ]:
mod_rndForest = TuningModel(mod_rndForest,parametros, X_train, y_train)

In [ ]:
mod_rndForest

In [ ]:
modelos = modelos.append({'modelo':mod_rndForest, 'scaler':False}  , ignore_index=True)

### EXTRA TREES CL

In [ ]:
mod_extrTrees = ExtraTreesClassifier (verbose=3, n_jobs=-1, random_state=14)
max_depth = [int(x) for x in np.linspace(5, 35, num = 10)]
max_depth.append(None)
params = {
    'n_estimators': [100,200],
    'max_depth': max_depth,
    'warm_start': ['True', 'False'],
    'ccp_alpha': [0,0.01, 0.05]
}

In [ ]:
mod_extrTrees= TuningModel(mod_extrTrees,parametros, X_train, y_train)

In [ ]:
mod_extrTrees

In [ ]:
modelos = modelos.append({'modelo':mod_extrTrees, 'scaler':False}  , ignore_index=True)

## TESTING MANUAL ENSEMBLES MODELS

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier, StackingClassifier, VotingClassifier

In [ ]:
modelos_ens = [('et', ExtraTreesClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=2)), 
            ('lr', LogisticRegression()),
            ('ab',AdaBoostClassifier(n_estimators=100, learning_rate=0.05)), 
            ('lgbm',LGBMClassifier(random_state=0, n_jobs=-1, learning_rate=0.05, n_estimators=100, max_depth=3))
]

### Voting Classifier

In [ ]:
mod_vc = VotingClassifier(estimators=modelos_ens,voting='soft')
modelos = modelos.append({'modelo':mod_vc, 'scaler':True}  , ignore_index=True)

### StackingEnsemble

In [ ]:
final_estimator = XGBClassifier(random_state=14, n_jobs=-1, learning_rate=0.05, n_estimators=100, max_depth=3)
Mod_stac_clss = StackingClassifier(estimators=modelos_ens, final_estimator=final_estimator)

modelos = modelos.append({'modelo':Mod_stac_clss, 'scaler':True}  , ignore_index=True)


### Bagging Classifer

In [ ]:
mod_bagg = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5)

modelos = modelos.append({'modelo':mod_bagg, 'scaler':False}  , ignore_index=True)

### GradientBoostingClassifier

In [ ]:
mod_gbc = GradientBoostingClassifier(learning_rate = 0.05, subsample=0.9, ccp_alpha= 0.015)
modelos = modelos.append({'modelo':mod_gbc, 'scaler':False}  , ignore_index=True)

## 3.3 EVALUATING MODELS

* Vstack is evaluated later because it transforms the training and test values

* TensorFlow is also evaluated separately


In [ ]:
#Checking all models are included
modelos

### Creating the EvalModelos Class
* This evaluates the models through the '***evaluar***' function. Returns a Dataframe object with these metrics:
    * Test / Training / CrossVal Accuracy
    * Standard Deviation of CV
    * Brier Loss
    * ROC-AUC
    * Training and prediction run times

* It also has three more functions:
    * get_models: returns the models that are passed to evaluation
    * set_param: It allows including a parameter dictionary in a selected model
    * get_param: Retrieves the parameters of the selected model
  
* This class can be used for one or more models

* When entering. the model is first evaluated if it needs to scale the data. If so, proceed to it first, otherwise directly evaluate the model.


In [ ]:
class EvalModelos:
 
  def __init__(self, modelos, X_train, y_train, X_test, y_test ):
    self.X_train = X_train
    self.X_test =  X_test
    self.y_train = y_train
    self.y_test =  y_test

    self.comparativa_modelos = pd.DataFrame(columns=['Modelo', 'Prec_test', 'Prec_train','Prec_cv',
                                                'ROC-AUC','STD_cv','Brier_loss','T_ejec_fit', 
                                                'T_ejec_pred'])
    self.modelos = modelos
    
  def evaluar (self):
    
    
    def EvalMod (modelo, X_train, y_train, X_test, y_test):
      print("Fitting modelo ", str(modelo).split('(')[0])
      kfold = KFold(5,shuffle=True)
      start = time.time()
      modelo.fit(X_train, y_train)
      t_ejec = time.time() - start
      print("predicting model ", str(modelo).split('(')[0])
      start_p = time.time()
      pred_mod = modelo.predict(X_test)
      prec_mod = accuracy_score(y_test, pred_mod)
      t_ejec_p = time.time() - start
      print("Starting CrossValidation", str(modelo).split('(')[0])
      prec_mod_cv = cross_val_score(modelo, X_train, y_train, cv=kfold).mean()
      std_prec_cv = statistics.stdev(cross_val_score(modelo, X_train, y_train))
      roc_auc = roc_auc_score(y_test, modelo.predict_proba(X_test)[:,1])
      prec_mod_train = modelo.score(X_train, y_train)
      brier_loss = brier_score_loss(y_test,modelo.predict_proba(X_test)[:,1] )
      return ({'Modelo':str(modelo).split('(')[0], 
                'Prec_test':prec_mod, 
                'Prec_train':prec_mod_train,
                'Prec_cv':prec_mod_cv,
                'ROC-AUC':roc_auc,
                'STD_cv':std_prec_cv,
                'Brier_loss':brier_loss,
                'T_ejec_fit':t_ejec,
                'T_ejec_pred':t_ejec_p
              }
              )
    #Checking if scaled values are needed  
    for mod in self.modelos.index:

      if self.modelos['scaler'][mod]==True:
        scaler = StandardScaler()
        X_train_sc = scaler.fit_transform(self.X_train)
        X_test_sc = scaler.transform(self.X_test)
        self.eval_modelo = EvalMod(self.modelos['modelo'][mod], X_train_sc, self.y_train, X_test_sc, self.y_test)
        self.comparativa_modelos = self.comparativa_modelos.append(self.eval_modelo, ignore_index=True)
      else:
        self.eval_modelo = EvalMod(self.modelos['modelo'][mod], self.X_train, self.y_train, self.X_test, self.y_test)
        self.comparativa_modelos = self.comparativa_modelos.append(self.eval_modelo, ignore_index=True)
      
    return self.comparativa_modelos

  ## get_model returns all models

  def get_models (self):
    return self.modelos

  ## set_params function sets parameters to models.
  #The model index and the parameter are passed in the form of a dict

  def set_param (self, index, params):
     ## Params must be a dictionary with the proper parameters,
     ## and index the index of the model within the list of models
    self.mod = self.modelos[index]
    for parameter, value in params.items():
        setattr( self.mod, parameter, value)
    return self.mod

  # get_params let us know the parameters of the models.
  
  def get_param (self, index):
     return self.modelos[index].get_params()

### Evaluating the models and add it to the comparativa_modelos DF

In [ ]:
start= time.time()
resultado = EvalModelos(modelos, X_train, y_train, X_test, y_test).evaluar()
comparativa_modelos = comparativa_modelos.append(resultado,ignore_index=True)
stop = time.time()

In [ ]:
print("Time to evaluate all models: ", (stop-start)/60, " Minutes")
comparativa_modelos

Now we are going to evaluate two models that require certain processes and cannot be integrated into the previous processes. The models are:

* **Tensorflow**
* **Vstack**

### 3.3.1 TENSORFLOW

### Model setting
* After several tests the best values ​​obtained are:

    * Three layers without dropout
    * MinMaxscaler as scaler instead of StandardScaler
    * We split the test block in two, one for validation and another for test to have an stop validation's earl stop in the fit.
    * CV is not generally used in neural networks as the efficiency of the network is mainly based on the data set, the number of epochs and the learning rate. In any case we are going to make a cv with KerasClassifier and including it in the comparision DF.



In [ ]:
model_tf = Sequential()
scaler_tf = MinMaxScaler()


y_train_tf = y_train.values
y_test_tf = y_test.values
X_train_tf = scaler_tf.fit_transform(X_train.values)
X_test_tf = scaler_tf.transform(X_test.values)

X_val_tf, X_test_2_tf, y_val_tf, y_test_2_tf = train_test_split(X_test_tf, y_test_tf, test_size=0.5, random_state=14)

earlstop = EarlyStopping(monitor='val_loss',mode='min', verbose=1, patience=20)


In [ ]:
model_tf.add(Dense(6, activation='relu'))
model_tf.add(Dense(12, activation='relu'))
model_tf.add(Dense(1, activation='sigmoid'))


### Compilation, fitting and evaluation of the model


In [ ]:
model_tf.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy',tf.metrics.AUC()])

CrossValidation with KerasClassifier

In [ ]:
def buildmodel():
    model= Sequential([
        Dense(6, activation='relu'),
        Dense(12, activation='relu'),
        Dense(1, activation='sigmoid')])
    model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics='accuracy')
    return model

estimator= KerasClassifier(build_fn=buildmodel, epochs=200, batch_size=250, verbose=1)
start=time.time()
kfold= kfold = KFold(5,shuffle=True, random_state=14)
results= cross_val_score(estimator, X_train_tf, y_train_tf, cv=kfold, n_jobs=-1, verbose=3)
print('Time CV: ',(time.time())-start, ' minuts')


In [ ]:
start = time.time()
history = model_tf.fit(X_train_tf, y_train_tf, 
                       verbose=1, batch_size=250,
                       callbacks=earlstop, epochs=600, 
                       validation_data=(X_val_tf, y_val_tf)
                       )
t_ejec = time.time() - start

start_p = time.time()
metrics = model_tf.evaluate(X_test_2_tf, y_test_2_tf)
metrics_tr = model_tf.evaluate(X_train_tf, y_train_tf)
metrics_val = model_tf.evaluate(X_val_tf, y_val_tf)
metrics_tr, metrics_val, metrics
t_ejec_p = time.time() - start_p



In [ ]:
std_prec_cv = statistics.stdev(results)

In [ ]:
"""--- SKIPPED CODE 
brier_loss = tfp.stats.brier_score(y_test_tf, model_tf.predict(X_test_tf), name=None)
brier_loss = np.array(brier_loss[:]).mean()

--- END CODE SKIPPED"""
##Brier loss can´t be evaluated in Kaggle due to TF version. 
#So in order to continue with the process I will assign the data calculated in the Colab notebook

In [ ]:
## The brier loss is negative in the Colab notebook. 
#It will be explained later but also here why it is accepted:
# If it were the Brier skill score then it would show that the model's fitted prediction 
#is not better than the prediction of the base model, but in the TF API a negative value 
#is considered normal. Reviewing the formula it is not the same as the Brier Kill Score 
#so I understand that it is a value that justifies the use of TF as an accurate model. 
#I have not found more information about it so any clarification that allows to confirm or 
#discard this approach will be apreciated. :-)
brier_loss = -0.005176

In [ ]:
valores = ({'Modelo':'TensorFlow', 
            'Prec_test':metrics[1], 
            'Prec_train':metrics_tr[1],
            'Prec_cv':results.mean(),
            'ROC-AUC':metrics_tr[2],
            'STD_cv': std_prec_cv,
            'Brier_loss': brier_loss,
            'T_ejec_fit':t_ejec,
            'T_ejec_pred':t_ejec_p})

In [ ]:
#Tensor Flow is added
comparativa_modelos = comparativa_modelos.append(valores,ignore_index=True)
comparativa_modelos

### 3.3.2 VSTACK ENSEMBLE

In [ ]:
!pip install vecstack

In [ ]:
from vecstack import stacking

In [ ]:
modelos = [
    ExtraTreesClassifier(random_state=0, n_jobs=-1, 
                         n_estimators=100, max_depth=2),
    LogisticRegression(),
    
    AdaBoostClassifier(n_estimators=100, learning_rate=0.05),
    
    LGBMClassifier(random_state=0, n_jobs=-1, learning_rate=0.05, 
                   n_estimators=100, max_depth=3),
    
]

In [ ]:
S_train, S_test = stacking(modelos,                     # list of models
                           X_train, y_train, X_test,   # data
                           regression=False,           # classification task (if you need 
                                                       #     regression - set to True)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and vote
                           needs_proba=False,          # predict class labels (if you need 
                                                       #     probabilities - set to True) 
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=accuracy_score,      # metric: callable
                           n_folds=4,                  # number of folds
                           stratified=False,            # stratified split for folds
                           shuffle=True,               # shuffle the data
                           random_state=14,             # ensure reproducibility
                           verbose=2)  

In [ ]:
model_Vstack = KNeighborsClassifier()
modelo = pd.DataFrame({'modelo':model_Vstack, 'scaler':True}, index=[0])

In [ ]:
eval_modelo = EvalModelos(modelo, S_train, y_train, S_test, y_test).evaluar()


In [ ]:
eval_modelo

As the name is the name of the last estimator, let`s chang it for Vstack

In [ ]:
eval_modelo['Modelo']='Vstack'
comparativa_modelos = comparativa_modelos.append(eval_modelo,ignore_index=True)
comparativa_modelos

# 4.TESTING THE BEST 3 MODELS ON UNSEEN DATA

  * Evaluation criterion:
    
    * Mean of (Prec_test /	Prec_cv ) >	Prec_train >	ROC-AUC >  Brier_loss >  STD_cv 	>	(T_ejec_fit 	T_ejec_pred)
  



* We assign a new column: '**media**' which will be the main evaluation criterion and will be the mean between the test's accuracy and the CV's accuracy.

In [ ]:
comparativa_modelos = comparativa_modelos.assign(media = lambda x: ((x['Prec_test']+x['Prec_cv']) /2 ))

* 6 models are choosen. As the ROC_AUC, the STD and the Brier Score are all at good levels, we will stick to making the selection of the top 3 based on training precision and execution times

  ** Note: ** The Tensorflow Brier Score is negative. If it were the Brier skill score then it would show that the model's fitted prediction is not better than the prediction of the base model, but in the TF API a negative value is considered normal. Reviewing the formula it is not the same as the Brier Kill Score so I understand that it is a value that justifies the use of TF as an accurate model. I have not found more information about it so any clarification that allows to confirm or discard this approach will be apreciated. :-)

In [ ]:
top6 = comparativa_modelos.sort_values(by='media', ascending=False)[:6]

In [ ]:
top6[['Modelo','media','Prec_train','T_ejec_pred','T_ejec_fit']]

With these data the chosen models are:
  1. **XGB**
  2. **SGDClassifier**
  3. **Logistic Regression**

 Now we proceed in this way:

  * First we fit with the entire training set and evaluate with the validation set we set apart and created with the 60% of the total combat's DF

  * Then we test a new fitting and evaluation with Crossvalidation with the validation set. The 60% of the total separated at the begining
  
  
***Notice that in google colab XGB is not in the first place but Tensorflow***

In [ ]:
df_combates_pokemons_train.info()

In [ ]:
df_combates_pokemons_valid.info()

In [ ]:
top3_eval = pd.DataFrame(columns=['Modelo', 'acc'])
model_top3 = [mod_xgb,mod_sgd,mod_lr]

We train the models with the entire training set and test it with the validation set

In [ ]:
## Scaling for SGD and LR
scaler = StandardScaler()

X = df_combates_pokemons_train.drop('Ganador',axis=1)
y = df_combates_pokemons_train['Ganador']

X_val = df_combates_pokemons_valid.drop('Ganador',axis=1)
y_val = df_combates_pokemons_valid['Ganador']

X_sc = scaler.fit_transform(X)
X_val_sc = scaler.transform(X_val)



In [ ]:
## Fitting models
mod_xgb.fit(X, y)
mod_sgd.fit(X_sc, y)
mod_lr.fit(X_sc,y)

In [ ]:
## Calculating the model's accuracy

y_p1 = mod_xgb.predict(X_val)
y_p2 = mod_sgd.predict(X_val_sc)
y_p3 = mod_lr.predict(X_val_sc)
acc1 = accuracy_score(y_val, y_p1)
acc2 = accuracy_score(y_val, y_p2)
acc3 = accuracy_score(y_val, y_p3)
top3_eval = top3_eval.append([{ 'Modelo':'XGB', 'acc':acc1},
                             {'Modelo':'SGD', 'acc':acc2},
                             {'Modelo':'Logistic', 'acc':acc3}])

In [ ]:
top3_eval 

CrossValidation on the validation set



In [ ]:
acc1= cross_val_score(mod_xgb, X_val, y_val, cv=5).mean()
acc2 = cross_val_score(mod_sgd, X_val_sc, y_val,cv=5).mean()
acc3 = cross_val_score(mod_lr,X_val_sc, y_val, cv=5).mean()

In [ ]:
top3_eval = top3_eval.assign(AccCv = [acc1,acc2, acc3] )

In [ ]:
top3_eval.assign(media = lambda x:(x['acc']+x['AccCv'])/2).sort_values(by='media', ascending=False)

The differences between all the models are minimal, however SGD execution times are much lower both in training and prediction, therefore our chosen model is **SGD**.

# 5 MODEL PRODUCTION

First we train the model with all the combats, the validation DF and the training DF.

Then we use two formats to evaluate it:
  1. Google Colab form
  2. Gradio

We also simulate as if it were a model put into production. In this way it should feed on each prediction made to incorporate it into the model and train it again with it. In addition, it should be monitored whether the Model is adjusting well. 

In order not to have to do fitting and prediction all the time, we do it every 30 predictions. **This is only available in Google Colab form not in Gradio**

The index for each Pokemon are in the original CSV.

**Note:** We assume that all functions and variables have been maintained throughout the process. otherwise, they would have to be loaded into the evaluation functions

In [ ]:
df_total = df_combates_pokemons_train.append(df_combates_pokemons_valid, ignore_index=True)
df_val = df_total[:1]
scaler = StandardScaler()
X_total = df_total.drop('Ganador',axis=1)
y_total = df_total['Ganador']
X_total_sc = scaler.fit_transform(X_total)
Count =1

In [ ]:
mod_sgd.fit(X_total_sc, y_total)
accuracy_score(y_total, mod_sgd.predict(X_total_sc))


### 5.1 FORM
Simulating a production environment using a Google Colab form

**As the form doesn´t shows as in colab the input is selected on the own cell and not in a slide. It should be run 31 times in order to show how the evaluation model works properly**

In [ ]:
#@title Evaluation using the Form { vertical-output: true }
#@markdown Select the pokemons using the slider and run the cell
pok1 = 85 #@param {type:"slider", min:0, max:1034, step:1}
pok2 = 452 #@param {type:"slider", min:0, max:1034, step:1}
def es_multiplo(numero, multiplo):
  return numero % multiplo == 0

def eval_mod (pok1, pok2):
  #Checking if 30 combats has been done
  if es_multiplo(Count, 30):
    X_total = df_total.drop('Ganador',axis=1)
    y_total = df_total['Ganador']
    X_total_sc = scaler.fit_transform(X_total)
    X_nuevos = df_val.drop('Ganador',axis=1)
    y_nuevos = df_val['Ganador']
    X_nuevos_sc = scaler.transform(X_nuevos)
    y_p = mod_sgd.predict(X_nuevos_sc) 
    # We monitor whether the model is still around 88.63% accuracy. We will put a fork of up to 88.45%
    # which we assume is acceptable and can be derived from the randomness of the processes and is similar to the std deviation.
    # We do it with the 30 new values.
    print("\n",'Monitoring model accuracy ')
    acc = accuracy_score (y_nuevos, y_p)
    if acc < 0.8845:
      print('The model is losing predictive capacity, now is :', np.round(acc,2), ' accuracy')
    else:
      print('Model is performing well')
    

    #We make a new fitting incorporating the 30 new predictions
    print("\n\n",'Fitting model again',"\n\n")
    mod_sgd.fit(X_total_sc, y_total)
    
  combat = Fight()
  combat = combat.combate(pok1, pok2)[0]
  
  X = combat.drop('Ganador', axis=1)
  print('Pok1 statistics',"\n\n",df.iloc[pok1] ,"\n\n",
      'Pok2 statistics',"\n\n",df.iloc[pok2] ,"\n\n",
      'Added combat stats: ',"\n\n", X,"\n\n")
  resultado_real = combat['Ganador'].values
  if resultado_real[0] == 0:
    res_real = 'Pokemon1'
  else:
    res_real ='Pokemon 2'
  y_p = mod_sgd.predict(X)
  if y_p == 0:
    pred = 'Pokemon 1'
  else:
    pred= 'Pokemon 2'
  pred_prob = mod_sgd.predict_proba(X)
  if y_p == resultado_real:
    val = '¡Bingo!'
  else:
    val= 'Mmm... failed'
  resultado = pd.DataFrame({'Prob Ganador Pok1': np.round(pred_prob[0][0],2),
                            'Prob ganador Pok2': np.round(pred_prob[0][1],2),
                           'Ganador predecido por modelo': pred,
                           'Ganador Real':res_real,
                           'acierto':val,
                            }, index=[0])
  
  
  
  return resultado, combat
agregado = eval_mod(pok1, pok2)
resultado = agregado[0]
combat = agregado[1]
df_total = df_total.append(combat, ignore_index=True)
df_val = df_val.append(combat, ignore_index=True)
if es_multiplo(Count, 30):
  df_val = df_val[:1]
Count +=1
print( 
      'Winner Pok1 prob: ', resultado['Prob Ganador Pok1'][0],"\n\n",
      'Winner Pok2 prob: ', resultado['Prob ganador Pok2'][0],"\n\n",
      'Model`s predicteed winner: ', resultado['Ganador predecido por modelo'][0],"\n\n",
      'Real Winner: ', resultado['Ganador Real'][0],"\n\n",
      'Model`s Performance: ', resultado['acierto'][0],"\n\n",
      'Total combats: ', Count)


### 5.2 GRADIO
In Gradio we do not update and monitor since the way to execute implies remaining in the launch call and is updated through Gradio's itself submit button, you do not have to rerun the cell so you cannot update the Count value.

**Moreover, the installing doesn´t work so it is only available in Google Colab. here it is obly show the code.**



In [ ]:
""" ---- SKIPPED CODE 
!pip install -q gradio

"""


In [ ]:
## import gradio as gr

In [ ]:
"""--- SKIPPED CODE 

def gradf (pok1, pok2):
  combat = Fight().combate(pok1, pok2)[0]
  X = combat.drop('Ganador', axis=1)
  resultado_real = combat['Ganador'].values
  if resultado_real[0] == 0:
    res_real = 'Pokemon1'
  else:
    res_real ='Pokemon 2'
  y_p = mod_sgd.predict(X)
  if y_p == 0:
    pred = 'Pokemon 1'
  else:
    pred= 'Pokemon 2'
  pred_prob = mod_sgd.predict_proba(X)
  if y_p == resultado_real:
    val = '¡Bingo'
  else:
    val= 'Mmm... falló'
  est_pok1 = pd.DataFrame(df.iloc[pok1]).T
  est_pok2 = pd.DataFrame(df.iloc[pok2]).T
  return df.iloc[pok1],df.iloc[pok2],X,round(pred_prob[0][0],2),round(pred_prob[0][1],2), pred, res_real, val



pok1 = gr.inputs.Slider(minimum=0, maximum=len(df), default=500, label="Indice Pokemon 1")
pok2 = gr.inputs.Slider(minimum=0, maximum=len(df), default=500, label="Indice Pokemon 2")
iface = gr.Interface(gradf,inputs=[pok1, pok2],
                     outputs=[gr.outputs.Textbox(label='Estadísticas Pokemon1'),
                              gr.outputs.Textbox(label='Estadísticas Pokemon2'),
                              gr.outputs.Dataframe(label='Estadísticas agregadas combate', type="pandas"),
                      gr.outputs.Textbox(label='Porcentaje victoria Pokemon1'),
                      gr.outputs.Textbox(label='Porcentaje victoria Pokemon2'),
                      gr.outputs.Textbox(label='Ganador predecido'),
                      gr.outputs.Textbox(label='Ganador real'),
                      gr.outputs.Textbox(label='Evaluación Modelo')],
                     title='Evaluación de modelo',
                     description= 'Veamos si el modelo acierta seleccionando un par de Pokemons de la base de datos')
iface.launch(debug=True, inbrowser =True)

--- END CODE SKIPPED """


# APPENDIX 1
* If it is wanted to tune and later evaluate all the models automatically we can do it here. It takes a long time in Google Collab but it is all automated
  * We create the tuning function and prepare the data
  * The EvalModelos function and the creation and separation of the database are carried out in the relevant modules (3.3 and 3.1) and must be executed there
  * TensorFlow and Vstack must be incorporated later



**AS IT LASTS A LOT OF TIME I JUST SHOW THE CODE BUT DON´T RUN IT TO AVOID A LONG TIME WHILE SAVING AND COMMITING**

In [ ]:
""" --- SKIPPED CODE 
comparativa_modelos = pd.DataFrame(columns=['Modelo', 'Prec_test', 'Prec_train','Prec_cv',
                                            'ROC-AUC','STD_cv','Brier_loss','T_ejec_fit', 
                                            'T_ejec_pred'])
--- END SKIPPED CODE"""

In [ ]:
""" --- SKIPPED CODE  
#Setting the matrix with the models to tune and the models to evaluate.
# The first ones will be included in the second ones after their tuning
modelos_tunning = pd.DataFrame(columns=['modelo','parametros','scaler'])
modelos_eval = pd.DataFrame(columns=['modelo','scaler'])

# We set the models to be assembled in VotingClassifier and StackingClassifier
modelos_ens = [('et', ExtraTreesClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=2)), 
            ('lr', LogisticRegression(C=0.5)),
            ('ab',AdaBoostClassifier(n_estimators=100, learning_rate=0.05)),
            ('lgbm',LGBMClassifier(random_state=0, n_jobs=-1, learning_rate=0.05, n_estimators=100, max_depth=2))
]
--- END SKIPPED CODE"""

In [ ]:
""" --- SKIPPED CODE 
# PREPARING THE MODELS TO EVALUATE
vecinos = np.arange(5,71,5)
mod1 = dict(modelo=KNeighborsClassifier(),
            parametros = {'n_neighbors':vecinos,
                          'weights': ['uniform','distance'],
                          'leaf_size':np.linspace(1,10,10).astype(int)
                          },
            scaler= True )
mod2 = dict(modelo= SGDClassifier(loss='modified_huber'),
            parametros = dict(penalty=['l1', 'l2', 'elasticnet'],
                              alpha=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
                              learning_rate=['constant', 'optimal', 'invscaling', 'adaptive'],
                              eta0=[1, 10, 100] ),  
            scaler= True )
mod3 = dict(modelo= LogisticRegression(),
            parametros = {'C': [ 1, 50, 100, 150], 
                          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
                          'max_iter':[100, 150,200]},
            scaler= False)
mod4 = dict(modelo= LGBMClassifier(objective='binary', n_estimators=150 ),
            parametros = {'num_leaves': [31,50,70],
              'learning_rate':[0.1,0.05,0.001],
              'max_depth':[-1,7,10,30]}, 
            scaler= False)
mod5 = dict(modelo= XGBClassifier(n_estimators=500),
            parametros = {'colsample_bylevel': [0.1,0.3,0.8,1],
              'colsample_bynode': [0.3,0.5,1],
              'gamma' : [0.05, 0.1, 0.2, 0.5, 1],
              'learning_rate': [0.005, 0.02,0.05],
              'max_depth': [1,2,4,6],
              'min_child_weight': [1,5,6,9],
              'subsample': [0.5,0.8,1]        
              }, 
            scaler= False)
mod6 = dict(modelo= AdaBoostClassifier(n_estimators=150, random_state=14),
            parametros = params = {'learning_rate': [0.05,0.1,0.5,0.8]},
            scaler= False )
max_depth = [int(x) for x in np.linspace(5, 35, num = 10)]
max_depth.append(None)
mod7 = dict(modelo=RandomForestClassifier(verbose=3, n_jobs=-1, random_state=14),
            parametros = {'n_estimators': [100, 200],
                          'max_depth': max_depth},
            scaler= False )
mod8 = dict(modelo= ExtraTreesClassifier (verbose=3, n_jobs=-1, random_state=14),
            parametros = {'n_estimators': [100,200],
                          'max_depth': max_depth,
                          'warm_start': ['True', 'False'],
                          'ccp_alpha': [0,0.01, 0.05]},
            scaler= False)
mod9 = dict(modelo= VotingClassifier(estimators=modelos_ens,voting='soft'),
            scaler= True )
final_estimator = XGBClassifier(random_state=14, n_jobs=-1,
                                learning_rate=0.05, n_estimators=100, max_depth=3)
mod10 = dict(modelo= StackingClassifier(estimators=modelos_ens, final_estimator=final_estimator),
            scaler= True )
mod11 = dict(modelo= BaggingClassifier(KNeighborsClassifier(), max_samples=0.5),
            scaler= True )
mod12 = dict(modelo= GradientBoostingClassifier(learning_rate = 0.05, subsample=0.9, ccp_alpha= 0.015),
            scaler= True)

##VSTACK Y TENSORFLOW ARE EVALUATING SEPARATELY
--- END SKIPPED CODE"""

In [ ]:
""" --- SKIPPED CODE 
modelos_tunning = modelos_tunning.append([mod1,mod2,mod3,mod4,mod5,mod6,mod7,mod8], ignore_index=True)
--- END SKIPPED CODE"""

In [ ]:
""" --- SKIPPED CODE 
def TunningAll (modelos_tunning, X_train, y_train, verbose=1, scoring='accuracy'):
  modelos = pd.DataFrame(columns=['modelo','scaler'])
  for id in modelos_tunning.index:    
      if modelos_tunning['scaler'][id]==True:
        mod_bayes_grid = BayesSearchCV(modelos_tunning['modelo'][id],modelos_tunning['parametros'][id], scoring= scoring, n_jobs=-1, verbose=verbose)
        scaler = StandardScaler()
        X_train_sc = scaler.fit_transform(X_train)
        mod_bayes_grid.fit(X_train_sc, y_train)
        modelos = modelos.append({'modelo':mod_bayes_grid.best_estimator_, 'scaler':True}, ignore_index=True)
      else:
        mod_bayes_grid = BayesSearchCV(modelos_tunning['modelo'][id],modelos_tunning['parametros'][id], scoring= scoring, n_jobs=-1, verbose=verbose)
        mod_bayes_grid.fit(X_train, y_train)
        modelos = modelos.append({'modelo':mod_bayes_grid.best_estimator_, 'scaler':False},ignore_index=True)
  return modelos
  --- END SKIPPED CODE"""


In [ ]:
""" --- SKIPPED CODE 
modelos_eval_tun = TunningAll(modelos_tunning, X_train, y_train)
--- END SKIPPED CODE"""

In [ ]:
""" --- SKIPPED CODE 
modelos_eval = modelos_eval.append([modelos_eval_tun, mod9, mod10,mod11, mod12],ignore_index=True)
--- END SKIPPED CODE"""

In [ ]:
""" --- SKIPPED CODE 
resultado = EvalModelos(modelos_eval, X_train, y_train, X_test, y_test).evaluar()
comparativa_modelos = comparativa_modelos.append(resultado,ignore_index=True)
comparativa_modelos
--- END SKIPPED CODE"""

AT THIS MOMENT, TENSORFLOW AND VSTACK WOULD BE ADDED AND WE WOULD HAVE THE WHOLE PACK OF MODELS WITH THEIR VALUES

# APPENDIX 2

In case we want to use the statistics generated in the battles to analyze the influence of randomness, we have to use the database generated in parallel in the Fight class.

Obviously we will never know the values ​​of those parameters but we can investigate how much it influences the final result.

We will use the winning model


In [ ]:
df_estadísticas_combate.info()

In [ ]:
df_estadísticas_combate.isna().sum()

In [ ]:
 df_estadísticas_combate.head(5)

We treat the NnN as zeros, since they are values ​​that have never been produced because the combat ends before the defending pokemon can attack. Therefore, there is neither the defender's attack nor the attacker's defense statistics.

If we would value these NaN as the mean or the mode, we are falsifying the test. It is not that we do not know the value and try to infer it, it is simply that it has not occurred.

On the other hand, when this situation occurs, the probability of succes is 100% since as soon as we know that the attack value of the defending Pokemon of the first turn is 0 and the defense of the attacking Pokemon in the first turn is 0, we know the Pokemon that attacked first in the first round surely won.


In [ ]:
X = df_estadísticas_combate.drop('Ganador', axis=1)
y = df_estadísticas_combate['Ganador']

In [ ]:
X = X.fillna(0)

In [ ]:
X.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=14)

In [ ]:
mod_sgd.fit(X_train, y_train)
y_p = mod_sgd.predict(X_test)
accuracy_score(y_test, y_p)

Accuracy is improved. In the event that we had the combat statistics prior to the prediction (Something impossible, that's why this appendix is ​​only for testing purposes) we could know with 92% who would be the winner. Almost 3 points more than without the statistics.

This means that the random statistics that are produced in the process of creating the combats are really very relevant and that randomness I guess is why I get an accuracy ceiling of 89%.

If I am misunderstanding something I can learn and someone can explain it, or there are another points of view, that would be apreciated to knonwn.